<a href="https://colab.research.google.com/github/abdullahsag/mobile-phone-review-using-Mistral-7B-Instruct-v0.1/blob/main/Mobile_Phone_Reviews_using_Mistral_7B_Instruct_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install in quiet mode (-q) and upgrade (-U) packages where necessary
!pip install -q -U langchain transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [5]:
llm = HuggingFacePipeline(pipeline=pipeline_inst)


In [40]:
template = """[INST] You are a knowledge technical writer, a Search Engine Optimizer and an expert of mobile phones. You will be provided with mobile phone specifications in tabular form and you will convert it into mobile phone reviews using conventional english.
Write a helpful 2000 word review optimized for search engines from context below :
{specifications} [/INST]
"""

In [41]:
prompt = PromptTemplate(template=template, input_variables=["specifications","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [42]:

def generate_response(specifications):
  # prompt = PromptTemplate(template=template, input_variables=["question","context"])
  # llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.invoke({"specifications":specifications})
  #print(response)
  # Split text based on [/INST]
  text_parts = response['text'].split('[/INST]')

  # Get the last part of the text
  last_part = text_parts[-1].strip()
  return last_part

In [43]:
specifications = """Mobile phone name: Oppo F25 Pro
Network
Technology	GSM / HSPA / LTE / 5G
2G bands	GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2
3G bands	HSDPA 900 / 2100
4G bands	1, 3, 5, 8, 28, 40, 41
5G bands	1, 3, 5, 8, 28, 40, 41, 77, 78 SA/NSA
Speed	HSPA, LTE, 5G
Display
Type	AMOLED, 1B colors, 120Hz, HDR10+, 500 nits (typ), 900 nits (HBM), 1100 nits (peak)
Size	6.7 inches, 108.0 cm2 (~89.8% screen-to-body ratio)
Resolution	1080 x 2412 pixels, 20:9 ratio (~394 ppi density)
Protection	Panda glass
Body
Dimensions	161.1 x 74.7 x 7.5 mm (6.34 x 2.94 x 0.30 in)
Weight	177 g (6.24 oz)
SIM	Dual SIM (Nano-SIM, dual stand-by) IP65 dust/water resistant
Memory
Card slot	microSDXC
Internal	128GB 8GB RAM, 256GB 8GB RAM UFS 3.1
Battery
Type	5000 mAh, non-removable
Charging	67W wired, PD2.0, QC3, 100% in 48 min (advertised) Reverse wired
Main Camera
Triple	64 MP, f/1.7, 25mm (wide), 1/2.0", 0.7µm, PDAF 8 MP, f/2.2, 16mm, 112˚ (ultrawide), 1/4.0", 1.12µm 2 MP, f/2.4, (macro)
Features	LED flash, HDR, panorama
Video	4K@30fps, 1080p@30/60/120/480fps, gyro-EIS
Selfie camera
Single	32 MP, f/2.4, 22mm (wide), 1/2.74", 0.8µm
Features	Panorama
Video	4K@30fps, 1080p@30fps, gyro-EIS
Comms
WLAN	Wi-Fi 802.11 a/b/g/n/ac/6, dual-band
Bluetooth	5.2, A2DP, LE, aptX HD
Positioning	GPS, GLONASS, GALILEO, BDS, QZSS
NFC	No
Radio	No
USB	USB Type-C 2.0, OTG
Infrared port	Yes
Platform
OS	Android 14, ColorOS 14
Chipset	Mediatek Dimensity 7050 (6 nm)
CPU	Octa-core (2x2.6 GHz Cortex-A78 & 6x2.0 GHz Cortex-A55)
GPU	Mali-G68 MC4
Features
Sensors	Fingerprint (under display, optical), accelerometer, gyro, proximity, compass
Sound
Loudspeaker	Yes
3.5mm jack	No
Misc
Colors	Ocean Blue, Lava Red
Launch
Announced	2024, February 29
Status	Available. Released 2024, March 04"""

In [44]:
response = generate_response(specifications=specifications)
response

'The Oppo F25 Pro is a smartphone that combines advanced technology and sleek design in a compact and portable package. The device is powered by Mediatek Dimensity 7050, making it a powerful and efficient device. It is equipped with a 5G network, allowing users to enjoy lightning-fast internet speeds and lag-free streaming of high-quality content.\n\nIn terms of display, the Oppo F25 Pro boasts an AMOLED panel with 1B colors and 120Hz, making it a great device for watching videos and playing games. The display also supports HDR10+, providing enhanced color accuracy and contrast. Additionally, the device has a resolution of 4032 \\* 1080 pixels, making it ideal for multitasking and browsing.\n\nOne of the standout features of the Oppo F25 Pro is its camera system. The device features a triple-lens setup with a 64 MP primary camera, making it an excellent choice for taking high-quality photos. It also has an 8 MP ultra-wide-angle lens and a 2 MP macro lens, providing versatility to suit 

In [45]:
response = generate_response("Who is the CEO of HuggingFace?")
response

"Hugging Face is a leading company in the field of natural language processing and artificial intelligence, and its CEO is Yuan Yao. Yao is an experienced executive with a background in technology and entrepreneurship. He has a proven track record of building successful companies in the field of AI, and has a passion for using technology to make the world a better place.\n\nIn addition to being CEO, Yao is also responsible for the overall direction and strategy of Hugging Face. He is constantly looking for new and innovative ways to apply the company's technology to real-world problems, and is committed to pushing the boundaries of what is possible in the field of AI.\n\nOne of Yao's key priorities is to make Hugging Face's technology accessible to as many people as possible. He believes that AI has the potential to transform many industries and improve the lives of countless people around the world, and he is committed to making that a reality.\n\nOverall, Yao is an inspiring and visi